In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.pipeline import Pipeline
from functions import *
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings


In [28]:
data = pd.read_csv('etfs_dataset_modelling.csv')
best_etfs = pd.read_csv('best_etfs.csv')
data = data[data.Ticker.isin(list(best_etfs['Ticker']))]
data

,Unnamed: 0,Ticker,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,RSI_9,RSI_14,RSI_25,ewma_7,ewma_50,ewma_200,Close_target
2149,2149,BNE,2020-12-09 00:00:00-05:00,24.820487,24.889597,24.435445,24.534174,27200.0,0.0,0.0,NaN,NaN,NaN,24.534174,24.534174,24.534174,24.425573
2150,2150,BNE,2020-12-10 00:00:00-05:00,24.218243,24.524303,24.218243,24.425573,25900.0,0.0,0.0,NaN,NaN,NaN,24.507024,24.529915,24.533093,24.286366
2151,2151,BNE,2020-12-11 00:00:00-05:00,24.336716,24.474938,24.237987,24.286366,4300.0,0.0,0.0,NaN,NaN,NaN,24.451859,24.520364,24.530638,24.405825
2152,2152,BNE,2020-12-14 00:00:00-05:00,24.603284,24.672394,24.405825,24.405825,1500.0,0.0,0.0,NaN,NaN,NaN,24.440351,24.515872,24.529396,25.294390
2153,2153,BNE,2020-12-15 00:00:00-05:00,24.583539,25.324007,24.583539,25.294390,10700.0,0.0,0.0,NaN,NaN,NaN,24.653860,24.546402,24.537008,25.347700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3570726,3570726,VRP,2024-07-03 00:00:00-04:00,23.910000,24.040001,23.910000,24.040001,281200.0,0.0,0.0,80.000305,86.478976,92.603276,23.938703,23.832645,23.363864,24.049999
3570727,3570727,VRP,2024-07-05 00:00:00-04:00,24.049999,24.070000,24.013000,24.049999,264600.0,0.0,0.0,60.869998,86.849410,92.906007,23.966527,23.841169,23.370691,24.070000
3570728,3570728,VRP,2024-07-08 00:00:00-04:00,24.070000,24.086000,24.031000,24.070000,220900.0,0.0,0.0,62.500298,88.972545,93.459252,23.992395,23.850142,23.377650,23.930000
3570729,3570729,VRP,2024-07-09 00:00:00-04:00,24.090000,24.100000,23.930000,23.930000,344500.0,0.0,0.0,65.384827,88.917000,92.944510,23.976797,23.853274,23.383146,24.030001


In [29]:
import pandas as pd
from datetime import timedelta

In [30]:
historic_data = data
historic_data

,Unnamed: 0,Ticker,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,RSI_9,RSI_14,RSI_25,ewma_7,ewma_50,ewma_200,Close_target
2149,2149,BNE,2020-12-09 00:00:00-05:00,24.820487,24.889597,24.435445,24.534174,27200.0,0.0,0.0,NaN,NaN,NaN,24.534174,24.534174,24.534174,24.425573
2150,2150,BNE,2020-12-10 00:00:00-05:00,24.218243,24.524303,24.218243,24.425573,25900.0,0.0,0.0,NaN,NaN,NaN,24.507024,24.529915,24.533093,24.286366
2151,2151,BNE,2020-12-11 00:00:00-05:00,24.336716,24.474938,24.237987,24.286366,4300.0,0.0,0.0,NaN,NaN,NaN,24.451859,24.520364,24.530638,24.405825
2152,2152,BNE,2020-12-14 00:00:00-05:00,24.603284,24.672394,24.405825,24.405825,1500.0,0.0,0.0,NaN,NaN,NaN,24.440351,24.515872,24.529396,25.294390
2153,2153,BNE,2020-12-15 00:00:00-05:00,24.583539,25.324007,24.583539,25.294390,10700.0,0.0,0.0,NaN,NaN,NaN,24.653860,24.546402,24.537008,25.347700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3570726,3570726,VRP,2024-07-03 00:00:00-04:00,23.910000,24.040001,23.910000,24.040001,281200.0,0.0,0.0,80.000305,86.478976,92.603276,23.938703,23.832645,23.363864,24.049999
3570727,3570727,VRP,2024-07-05 00:00:00-04:00,24.049999,24.070000,24.013000,24.049999,264600.0,0.0,0.0,60.869998,86.849410,92.906007,23.966527,23.841169,23.370691,24.070000
3570728,3570728,VRP,2024-07-08 00:00:00-04:00,24.070000,24.086000,24.031000,24.070000,220900.0,0.0,0.0,62.500298,88.972545,93.459252,23.992395,23.850142,23.377650,23.930000
3570729,3570729,VRP,2024-07-09 00:00:00-04:00,24.090000,24.100000,23.930000,23.930000,344500.0,0.0,0.0,65.384827,88.917000,92.944510,23.976797,23.853274,23.383146,24.030001


In [31]:
#def predict_future_prices(historic_data, days_ahead):
    # Preparación de datos históricos

historic_data = clean_data(historic_data)

X = [historic_data for historic_data in historic_data.groupby('Ticker')]
y = [historic_data for historic_data in historic_data.groupby('Ticker')['Close_target']]

future_df = [] #lista de dataframes por ticker con fechas futuras

for i in range(len(X)):
    # Aseguramos que las fechas estan ordenadas
    etf = X[i][1].sort_values(by = 'Date')

    # Última fecha conocida
    last_date = etf['Date'].max()

    # Generar las próximas 30 fechas (en el futuro modelo será un N que elija el usuario)
    future_dates = [last_date + timedelta(days=i) for i in range(1,91)]
    
    # Crear un DataFrame para las fechas futuras
    etf_df = pd.DataFrame({'Date':future_dates, 'Ticker': X[i][0]})
    #print(etf_df)
    etf = pd.concat([etf_df, X[i][1]]).sort_values(by='Date')
    #print(etf)
    future_df.append(etf)

future_df = pd.concat(future_df)

future_df


,Date,Ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,RSI_9,...,ewma_50,ewma_200,Close_target,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
3309449,2018-02-07,AADR,58.441951,59.092072,58.038084,58.146439,39100.0,0.0,0.0,8.473503,...,60.187097,59.701113,55.772511,57.810001,57.540001,57.820000,57.430000,56.959999,56.090000,56.430000
3309450,2018-02-08,AADR,58.205541,58.599555,55.585356,55.772511,86700.0,0.0,0.0,2.818188,...,60.013976,59.662022,56.028625,58.146439,57.810001,57.540001,57.820000,57.430000,56.959999,56.090000
3309451,2018-02-09,AADR,56.146828,56.462038,54.166915,56.028625,85900.0,0.0,0.0,0.000000,...,59.857688,59.625869,57.516026,55.772511,58.146439,57.810001,57.540001,57.820000,57.430000,56.959999
3309452,2018-02-12,AADR,56.580244,57.772136,56.580244,57.516026,100500.0,0.0,0.0,0.000000,...,59.765858,59.604875,57.348568,56.028625,55.772511,58.146439,57.810001,57.540001,57.820000,57.430000
3309453,2018-02-13,AADR,57.417520,57.575125,57.062908,57.348568,31900.0,0.0,0.0,0.000000,...,59.671062,59.582425,58.914768,57.516026,56.028625,55.772511,58.146439,57.810001,57.540001,57.820000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2024-10-03,ZHDG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,2024-10-04,ZHDG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,2024-10-05,ZHDG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,2024-10-06,ZHDG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
tickers = future_df['Ticker'].unique()

future_df = future_df.fillna(calculate_ewma(future_df,tickers,[7,50,200]))


g:\ID Bootcamps - Data Science & ML\ETF_project\db\functions.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ticker[f'ewma_{num}'] = df_ticker['Close'].ewm(span=num, adjust=False).mean()


In [33]:
future_df.columns

Index(['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'RSI_9', 'RSI_14', 'RSI_25', 'ewma_7', 'ewma_50',
       'ewma_200', 'Close_target', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5',
       'lag_6', 'lag_7'],
      dtype='object')

In [34]:
future_df.to_csv('future_df.csv')

In [35]:
future_data = pd.read_csv('future_df.csv')
future_data[future_data['Ticker'] == 'BNE']# vemos que los lags llegan bien para cada etf

,Unnamed: 0,Date,Ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,...,ewma_50,ewma_200,Close_target,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
143038,2181,2021-01-27,BNE,31.815193,32.387904,31.299750,31.485388,6900.0,0.0,0.0,...,28.409158,25.806623,31.613752,32.891499,32.665375,32.907295,32.648586,31.834942,31.779644,30.472279
143039,2182,2021-01-28,BNE,31.657202,31.686824,31.351096,31.613752,3300.0,0.0,0.0,...,28.534828,25.864405,30.518688,31.485388,32.891499,32.665375,32.907295,32.648586,31.834942,31.779644
143040,2183,2021-01-29,BNE,31.173358,31.193108,30.354773,30.518688,11500.0,0.0,0.0,...,28.612627,25.910716,31.334311,31.613752,31.485388,32.891499,32.665375,32.907295,32.648586,31.834942
143041,2184,2021-02-01,BNE,31.015369,31.400468,30.610520,31.334311,8000.0,0.0,0.0,...,28.719360,25.964682,31.728296,30.518688,31.613752,31.485388,32.891499,32.665375,32.907295,32.648586
143042,2185,2021-02-02,BNE,31.617704,31.943556,31.351096,31.728296,5200.0,0.0,0.0,...,28.837357,26.022032,32.299034,31.334311,30.518688,31.613752,31.485388,32.891499,32.665375,32.907295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143990,85,2024-10-03,BNE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.457200,22.493588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143991,86,2024-10-04,BNE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.457200,22.493588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143992,87,2024-10-05,BNE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.457200,22.493588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143993,88,2024-10-06,BNE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.457200,22.493588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Tenemos que predecir los datos faltantes

In [38]:
def extra_tree_model_future(data_past_future):

    mse_etfs = dict()
    r2_etfs = dict()
    predictions = dict()
    dates_pred = dict()
    models = dict()

    # Trabajamos con datos historicos
    
    X = [data_past_future for data_past_future in data_past_future.groupby('Ticker')]
    y = [data_past_future for data_past_future in data_past_future.groupby('Ticker')['Close_target']]
    
    for i in range(len(X)):

        X = X[i][1].iloc[:-90]
        X['Pct_day_change'] = X['Pct_day_change'].fillna(method = 'ffill')
        y = y[i][1].iloc[:-90]
        
        # seleccionamos todo el dataframe menos las 90 ultimas filas (datos futuros)
        historical_data_X = X
        historical_data_y = y
    
        
        prueba_X = historical_data_X.drop(['Unnamed: 0','Ticker','Close_target', 'Date', 'Volume', 'Dividends', 'Stock Splits', 'RSI_9', 'RSI_14', 'RSI_25'], axis=1)
        prueba_y = historical_data_y


        X_train, y_train, X_val, y_val, X_test, y_test = data_train_test_regression(prueba_X, prueba_y)

        print('nulos Xtrain:', X_train.isnull().sum())
        extr_model = ExtraTreesRegressor(max_depth= 10, 
                                        max_samples= None, 
                                        min_samples_split= 5, 
                                        n_estimators= 200, 
                                        random_state=42)

        extr_model.fit(X_train, y_train)
        y_pred_val = extr_model.predict(X_val)

        mse_extr = mean_squared_error(y_val, y_pred_val)
        r2_extr = r2_score(y_val, y_pred_val)
        print(f'Ticker {X[i][0]} {i} validation:\n')
        print(f'MSE val ExtraTree: {mse_extr}')
        print(f'R2 ExtraTree: {r2_extr}\n')


        X_combined = pd.concat([X_train, X_val])
        y_combined = pd.concat([y_train, y_val])

        
        extr_model.fit(X_combined, y_combined)
        y_pred = extr_model.predict(X_test)

        predictions[X[i][0]] = y_pred

        mse_extr_test = mean_squared_error(y_test, y_pred)
        r2_extr_test = r2_score(y_test, y_pred)
        print(f'Ticker {X[i][0]} {i} test:\n')
        print(f'MSE ExtraTree: {mse_extr_test}')
        print(f'R2 ExtraTree: {r2_extr_test}\n')
        print('***********''\n')

        # recogemos todas nuestras metricas
        mse_etfs[X[i][0]] = mse_extr_test
        r2_etfs[X[i][0]] = r2_extr_test

        # Guardar el modelo entrenado
        models[X[i][0]] = extr_model

        

    return predictions, mse_etfs, r2_etfs, dates_pred, models

In [39]:
predictions, mse_etfs, r2_etfs, dates_pred, future_predictions = extra_tree_model_future(future_data)

TypeError: list indices must be integers or slices, not str

In [ ]:
 # Trabajamos con datos futuros (90 días)
    future_predictions = dict()

    for ticker in models.keys():
        # Seleccionar los próximos 90 días del dataframe future_data
        future_df = data_past_future[data_past_future['Ticker'] == ticker].tail(90)
        
        # Eliminar las mismas columnas que en 'prueba_X'
        future_df_features = future_df.drop(['Unnamed: 0', 'Date', 'Ticker', 'Open', 'High', 'Low', 'Close',
                                            'Volume', 'Dividends', 'Stock Splits', 'RSI_9', 'RSI_14', 'RSI_25', 
                                            'lag_1', 'lag_2','lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'Pct_day_change'], axis=1)
        

        # Predecir con el modelo guardado
        future_pred = models[ticker].predict(future_df_features)
        
        # Guardar las predicciones
        future_predictions[ticker] = future_pred